# Application: Creating Dense City-Scale Traffic Models

Copyright © Scott Workman. 2024.

In [ ]:
import math
import numpy as np
import osmnx as ox
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

ox.settings.log_console = True
ox.settings.use_cache = True
ox.__version__

## Setup

load results produced by extract/extract_speeds.py

In [ ]:
df = pd.read_pickle('results.pkl')

generate the street network

In [ ]:
place = 'Brooklyn, NY, USA'

G = ox.graph_from_place(place, network_type='drive')
G = ox.project_graph(G)
G_tar = G.copy()

create lookup table (segment -> speed)

In [ ]:
edge_lookup_pred = {}
edge_lookup_tar = {}

for index, row in df.iterrows():
  start = row['osmstartnode']
  end = row['osmendnode']
  if (start == "nan") or (end == "nan"):
    continue
  edge_lookup_pred[(int(start), int(end))] = row['prediction']
  edge_lookup_tar[(int(start), int(end))] = row['target']

print(len(edge_lookup_pred.keys()))

edges weighted by travel time

In [ ]:
meters_per_minute = lambda x: x * 1000 / 60  # km/h to m/m

def compute_edge_values(G, edge_lookup, default_speed):
  edge_values = []
  to_remove = []

  for u, v, data in G.edges(data=True):
    if (u, v) in edge_lookup.keys():
      speed = edge_lookup[(u, v)]
    else:
      speed = default_speed

    if math.isnan(speed):
      to_remove.append((u, v))
      continue

    edge_values.append(np.log(meters_per_minute(speed)))

  for u, v in to_remove:
    G.remove_edge(u, v)

  return edge_values

In [ ]:
edge_values_pred = compute_edge_values(G, edge_lookup_pred,
                                       df['prediction'].mean())
edge_values_tar = compute_edge_values(G_tar, edge_lookup_tar,
                                      df['target'].mean())

## Visualize

In [ ]:
vmin = 5.5
vmax = 6.8

norm = colors.Normalize(vmin=vmin, vmax=vmax)
cmap = cm.ScalarMappable(norm=norm, cmap='RdYlGn')

### Prediction

In [ ]:
edge_colors = [cmap.to_rgba(cl) for cl in edge_values_pred]

fig, ax0 = ox.plot_graph(G,
                         bgcolor='w',
                         show=False,
                         close=False,
                         node_size=0,
                         node_color='k',
                         node_edgecolor='gray',
                         node_zorder=2,
                         node_alpha=0.9,
                         edge_color=edge_colors,
                         edge_linewidth=2.5,
                         edge_alpha=0.9,
                         figsize=(12, 12))
fig, ax = ox.plot_graph(G,
                        bgcolor='w',
                        edge_color="k",
                        edge_linewidth=1,
                        edge_alpha=0.35,
                        node_size=0,
                        node_color='none',
                        ax=ax0,
                        figsize=(12, 12))

### Ground Truth

In [ ]:
edge_colors = [cmap.to_rgba(cl) for cl in edge_values_tar]

fig, ax0 = ox.plot_graph(G_tar,
                         bgcolor='w',
                         show=False,
                         close=False,
                         node_size=0,
                         node_color='k',
                         node_edgecolor='gray',
                         node_zorder=2,
                         node_alpha=0.9,
                         edge_color=edge_colors,
                         edge_linewidth=2.5,
                         edge_alpha=0.9,
                         figsize=(12, 12))
fig, ax = ox.plot_graph(G_tar,
                        bgcolor='w',
                        edge_color="k",
                        edge_linewidth=1,
                        edge_alpha=0.35,
                        node_size=0,
                        node_color='none',
                        ax=ax0,
                        figsize=(12, 12))